In [1]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.dynamicframe import DynamicFrame
from awsglue.job import Job
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
glueContext = GlueContext(SparkContext.getOrCreate())
spark = glueContext.spark_session

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1556550105484_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
pg_dyf = glueContext.create_dynamic_frame.from_catalog(database = "testcollins", table_name = "small_collinsdb_public_small_flights_tyler")

In [4]:
s3_dyf = glueContext.create_dynamic_frame.from_catalog(database = "testcollins", table_name = "s3small_smallflight")

In [5]:
pg_df = pg_dyf.toDF()
s3_df = s3_dyf.toDF()

In [6]:
pg_df.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+

In [7]:
s3_df.show()

+-----------+--------------------+---------+-------+----------+--------+
|messageType|               route|departure|arrival|tailNumber|flightId|
+-----------+--------------------+---------+-------+----------+--------+
|        FPL|NIITE3 SYRAH Q130...|     KSFO|   KDFW|    0E640B| 8C47211|
+-----------+--------------------+---------+-------+----------+--------+

In [9]:
pg_df[pg_df['flight_id'] == 'FLIGHTIDABC'].show()

+-----------+---------------+---------+-------+-----------+------------+
|  flight_id|          route|departure|arrival|tail_number|message_type|
+-----------+---------------+---------+-------+-----------+------------+
|FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
+-----------+---------------+---------+-------+-----------+------------+

In [10]:
columns = ['id', 'dogs', 'cats']
vals = [(1, 2, 0), (2, 0, 1)]

df = spark.createDataFrame(vals, columns)

In [12]:
newRow = spark.createDataFrame([(4,5,7)], columns)
appended = df.union(newRow)
appended.show()

+---+----+----+
| id|dogs|cats|
+---+----+----+
|  1|   2|   0|
|  2|   0|   1|
|  4|   5|   7|
+---+----+----+

In [17]:
def squared(s):
  return s * s
spark.udf.register("squaredWithPython", squared)
from pyspark.sql.types import LongType
def squared_typed(s):
  return s * s
spark.udf.register("squaredWithPython", squared, LongType())

In [23]:
pg_cols = ['flight_id', 'route', 'departure', 'arrival', 'tail_number', 'message_type']
pg_new_vals = [('LMNFLIGHTID', 'KLAS KASE UF', 'MSNBC', 'RT', '3030G', 'NOT NULL')]
newRow3 = spark.createDataFrame([('LMNFLIGHTID', 'KLAS KASE UF', 'MSNBC', 'RT', '3030G', 'NOT NULL')], pg_cols)


In [24]:
appended2 = pg_df.union(newRow3)

In [25]:
appended2.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
|        LMNFLIGHTID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+

In [29]:
pg_cols2 = ['flight_id', 'route', 'departure', 'arrival', 'tail_number', 'message_type']
pg_new_vals2 = [('OPFLIGHID', 'KLAS KASE UF', 'MSNBC', 'RT', '3030G', 'NOT NULL')]
newRow4 = spark.createDataFrame([('OPFLIGHID', 'KLAS KASE UF', 'MSNBC', 'RT', '3030G', 'NOT NULL')], pg_cols)
appended3 = appended2.union(newRow4)

In [30]:
appended3.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
|        LMNFLIGHTID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
|          OPFLIGHID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+

In [31]:
appended3.createOrReplaceTempView("flights_temp_table")

In [34]:
sqlTemp = spark.sql("SELECT * FROM flights_temp_table")

In [36]:
sqlTemp.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
|        LMNFLIGHTID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
|          OPFLIGHID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+

In [37]:
pg_cols3 = ['flight_id', 'route', 'departure', 'arrival', 'tail_number', 'message_type']
newRow5 = spark.createDataFrame([('OPFLIGHID', 'KLAS KASE UF', 'MSNBC', 'RT', '3030G', 'NOT NULL')], pg_cols)
appended4 = appended3.union(newRow5)

In [38]:
appended4.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
|        LMNFLIGHTID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
|          OPFLIGHID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
|          OPFLIGHID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+

In [42]:
noop = appended4.filter(appended4.flight_id != 'OPFLIGHID')

In [43]:
noop.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
|        LMNFLIGHTID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+

In [54]:
uniq_df = appended4.select('flight_id').distinct().collect()

In [58]:
uniq2 = appended4.dropDuplicates(['flight_id'])

In [59]:
uniq2.show()

+-------------------+---------------+---------+-------+-----------+------------+
|          flight_id|          route|departure|arrival|tail_number|message_type|
+-------------------+---------------+---------+-------+-----------+------------+
|        FLIGHTIDABC|KD45Q DCT KD42S|      CNN|    FOX|      6969B|    NOT NULL|
|        LMNFLIGHTID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
|          OPFLIGHID|   KLAS KASE UF|    MSNBC|     RT|      3030G|    NOT NULL|
|        AVCFLIGHTID| HASSL DCT KD45|    MSNBC|     RT|     56556C|    NOT NULL|
|FOURFIVESIXFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      0086H|    NOT NULL|
| ONETOTHREEFLIGHTID|    OOP UUY HYN|      HOS|    RBC|      7878T|    NOT NULL|
+-------------------+---------------+---------+-------+-----------+------------+